In [1]:
#import sys
#sys.path.insert(0, "/home/martial/Data/programs/Github/hyperspy_T/")
import hyperspy.api as hs

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
%matplotlib qt5

In [4]:
input_file_name = '../../Demo_data/SAED_02'
input_file_extension = '.dm4'
data_file = hs.load(input_file_name+input_file_extension)

In [5]:
data_file.plot()

In [8]:
data_file.axes_manager

Signal axis name,size,offset,scale,units
x,4096,-0.0,0.010874038562178612,1/nm
y,4096,-0.0,0.010874038562178612,1/nm


In [ ]:
#initialize the center coordinates in signal units
Center_coor = (21.26, 22.45)
Center_coor[0]

In [ ]:
#Initilize data average filled with 0 values.
data_average = hs.signals.Signal1D(np.zeros((np.int(data_file.data.shape[0])),dtype=float))
data_average.axes_manager[0].scale = data_file.axes_manager[0].scale
data_average.axes_manager[0].units = data_file.axes_manager[0].units

In [ ]:
data_average.axes_manager

In [ ]:
# calculate the rotational average in k-space (1/nm units)
Center_coor_px = (np.int(Center_coor[0]/data_file.axes_manager[0].scale),\
                  np.int(Center_coor[1]/data_file.axes_manager[0].scale))

for x in range( data_file.axes_manager[0].size ):
    for y in range( data_file.axes_manager[1].size ):
        data_average.data[np.int(np.sqrt((x-Center_coor_px[0])**2+(y-Center_coor_px[1])**2))]= \
        +data_average.data[np.int(np.sqrt((x-Center_coor_px[0])**2+(y-Center_coor_px[1])**2))]\
        +data_file.data[x,y]

In [ ]:
#Only the k-space only goes up to data_file size/2 (as k=0 is about at the center of the image)
#Can be improved by considering the center of the SAED
data_average.crop_signal1D([0,np.int(data_file.data.shape[0]/2)])

In [ ]:
lambda_Xray = 0.154059 #nm for 200kV

In [ ]:
#data_average.save('data_average_pos009.hdf5')
#data_average = hs.load('data_average_pos009.hdf5')

In [ ]:
data_average.plot()

In [ ]:
data_average_2theta = hs.signals.Signal1D(np.zeros(((2,np.int(data_file.data.shape[0]))),dtype=float))
#initilize data_average_2theta so data_average_2theta now is like empty with only zero values. 

In [ ]:
data_average_2theta.axes_manager[1].scale = 0.05
data_average_2theta.axes_manager[1].units = '2_theta'

In [ ]:
data_average_2theta.axes_manager

In [ ]:
# Transform the k-space coordinates in 2_theta coordinates

for x in range( data_average.axes_manager[0].size ):
    k_x = (x+data_average.axes_manager[0].offset)*data_average.axes_manager[0].scale
    if(k_x*lambda_Xray/2<1):
        theta_2_x = 2*np.arcsin(k_x*lambda_Xray/2)*(180/np.pi)
    elif(theta_2_x>data_average_2theta.data.shape[0]):
        break
    else:
        break
    data_average_2theta.data[0,np.int(np.round(theta_2_x/data_average_2theta.axes_manager[1].scale))]\
    =data_average_2theta.data[0,np.int(np.round(theta_2_x/data_average_2theta.axes_manager[1].scale))]\
    +data_average.data[x]
    data_average_2theta.data[1,np.int(np.round(theta_2_x/data_average_2theta.axes_manager[1].scale))]\
    =data_average_2theta.data[1,np.int(np.round(theta_2_x/data_average_2theta.axes_manager[1].scale))]\
    +1

for x in range(data_average_2theta.axes_manager[0].size ):
    if(data_average_2theta.data[1,x]!=0):
        data_average_2theta.data[0,x]=data_average_2theta.data[0,x]/data_average_2theta.data[1,x]

In [ ]:
data_average_2theta.crop_signal1D(left_value=6., right_value=70.)
#get data ranges from 6 to 70 only crop of the [0] or the [1] axis 

for x in range(data_average_2theta.axes_manager[1].size ):
    if(data_average_2theta.data[0,x]<0.1):
        if(data_average_2theta.data[0,x+1]>0.1):
            data_average_2theta.data[0,x]=(data_average_2theta.data[0,x-1]+data_average_2theta.data[0,x+1])/2
        elif(data_average_2theta.data[0,x+2]>0.1):
            data_average_2theta.data[0,x]=(data_average_2theta.data[0,x-1]+data_average_2theta.data[0,x+2])/2
        else:
            print(x)

In [ ]:

for x in range(data_average_2theta.axes_manager[1].size ):
    if(data_average_2theta.data[0,x]<0.1):
        j=0
        while((x+j)<(data_average_2theta.axes_manager[1].size-1) and data_average_2theta.data[0,x+j]<0.1):
            #condition: x is not out of range and the data at [0,x+j]<0.1
            if (x+j<data_average_2theta.axes_manager[1].size):
                data_average_2theta.data[0,x]= \
                (data_average_2theta.data[0,x-1]+data_average_2theta.data[0,x+j+1])/2
            else:
                while (x<data_average_2theta.axes_manager[1].size):
                    data_average_2theta.data[0,x]=data_average_2theta.data[0,x-1]
                    x=x+1
                break
            j=j+1
        x=x+j
# why do we have to average like this? 

In [ ]:
data_average_2theta_sum = hs.signals.Signal1D( \
    np.zeros(((np.int(data_average_2theta.axes_manager[1].size))),dtype=float))
data_average_2theta_sum.axes_manager[0].scale = data_average_2theta.axes_manager[1].scale
data_average_2theta_sum.axes_manager[0].units = data_average_2theta.axes_manager[1].units
data_average_2theta_sum.axes_manager[0].offset = data_average_2theta.axes_manager[1].offset

In [ ]:
data_average_2theta_sum.data[:] = data_average_2theta.data[0,:]

In [ ]:
data_average_2theta_sum.plot()

In [ ]:
#data_average_2theta_sum.save(input_file_name+'.hdf5')

In [ ]:
#### Plot XRD lines calculated from Vesta

In [ ]:
XRD_4124388 = hs.signals.Signal1D(np.zeros(((2,500)),dtype=float))
with open('4124388.txt') as f:
    data = f.readlines()

i=0
for line in data:
    words = line.split()
    if (i>0):
        XRD_4124388.data[0,i] = np.float(words[7])
        XRD_4124388.data[1,i] = np.float(words[8])
    i=i+1
#what is words[7]?

In [ ]:
XRD_PbI2 = hs.signals.Signal1D(np.zeros(((2,500)),dtype=float))
with open('PbI2_P-3m1.txt') as f:
    data = f.readlines()

i=0
for line in data:
    words = line.split()
    if (i>0):
        XRD_PbI2.data[0,i] = np.float(words[7])
        XRD_PbI2.data[1,i] = np.float(words[8])
    i=i+1

In [ ]:
theta_2_x

In [ ]:
theta_2_x = np.zeros(data_average_2theta_sum.data.size, float)
theta_2_x = np.linspace(data_average_2theta_sum.axes_manager[0].offset, \
            #data_average_2theta_sum.axes_manager[0].scale*data_average_2theta_sum.axes_manager[0].size \
            0.051*data_average_2theta_sum.axes_manager[0].size \
            +data_average_2theta_sum.axes_manager[0].offset, \
            data_average_2theta_sum.data.size, float)
#generating theta 2x from offset to 0.051*size+offset with the number of points = size

plt.plot(theta_2_x[:], data_average_2theta_sum.data[:]/np.max(data_average_2theta_sum.data[:]), \
         'bo', ms =1)
#why do we need to divide by the max? 
#this is to draw the line at?
x=0
while(XRD_4124388.data[0,x]<theta_2_x.data[-1]):
    plt.axvline(x=XRD_4124388.data[0,x],ymax=XRD_4124388.data[1,x]/np.max(XRD_4124388.data[1,:]),\
                color='blue')
    x=x+1
    
x=0
while(XRD_PbI2.data[0,x]<theta_2_x.data[-1]):
    plt.axvline(x=XRD_PbI2.data[0,x],ymax=XRD_PbI2.data[1,x]/np.max(XRD_PbI2.data[1,:]),\
                color='red')
    x=x+1
    
plt.savefig( input_file_name +".png", dpi=150)